<a href="https://colab.research.google.com/github/ilkelity/Hands-on-Machine-Learning-with-Scikit-Learn-Keras-and-TensorFlow/blob/master/HW2_Hybrid_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

In [10]:
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [11]:
!kaggle datasets download -d grouplens/movielens-20m-dataset

movielens-20m-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
!unzip \*.zip

unzip:  cannot find or open *.zip, *.zip.zip or *.zip.ZIP.

No zipfiles found.


In [14]:
import pandas as pd
pd.set_option('display.max_columns', 20)

movie = pd.read_csv("/content/drive/MyDrive/Kaggle/movie.csv")
movie.head()
movie.shape

(27278, 3)

In [15]:
ratings = pd.read_csv("/content/drive/MyDrive/Kaggle/rating.csv")
ratings.shape


(20000263, 4)

In [16]:
ratings.head(15)

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40
5,1,112,3.5,2004-09-10 03:09:00
6,1,151,4.0,2004-09-10 03:08:54
7,1,223,4.0,2005-04-02 23:46:13
8,1,253,4.0,2005-04-02 23:35:40
9,1,260,4.0,2005-04-02 23:33:46


In [17]:
df = movie.merge(ratings, how="left", on="movieId")

In [18]:
df.tail(15)

,movieId,title,genres,userId,rating,timestamp
20000782,131176,A Second Chance (2014),Drama,109286.0,4.5,2015-03-30 05:10:03
20000783,131180,Dead Rising: Watchtower (2015),Action|Horror|Thriller,117144.0,2.5,2015-03-30 07:17:52
20000784,131231,Standby (2014),Comedy|Romance,63046.0,3.5,2015-03-30 14:04:06
20000785,131237,What Men Talk About (2010),Comedy,134701.0,3.0,2015-03-30 17:14:50
20000786,131239,Three Quarter Moon (2011),Comedy|Drama,79570.0,4.0,2015-03-30 18:03:09
20000787,131241,Ants in the Pants (2000),Comedy|Romance,79570.0,4.0,2015-03-30 18:12:10
20000788,131243,Werner - Gekotzt wird später (2003),Animation|Comedy,79570.0,4.0,2015-03-30 18:17:10
20000789,131248,Brother Bear 2 (2006),Adventure|Animation|Children|Comedy|Fantasy,79570.0,4.0,2015-03-30 19:04:48
20000790,131250,No More School (2000),Comedy,79570.0,4.0,2015-03-30 19:16:42
20000791,131252,Forklift Driver Klaus: The First Day on the Jo...,Comedy|Horror,79570.0,4.0,2015-03-30 19:20:55


<h2 style="background-color:powderblue;">HYBRID Recommender</h2>

item based/memory based

ürünlerin benzerliği üzerinden öneriler yapılır

puan benzerlikleri üzerinden öneriler yapılır

In [19]:
# user-movie matrisinin oluşturulması
# öneri yapmadan önce bazı filtrelemeler yapmak uygun olacaktır, mesela çok düşük puanlı olan ürünlerin elenmesi


In [20]:
df["title"].value_counts().head()  # her bir filme yapılan değerlendirme

Pulp Fiction (1994)                 67310
Forrest Gump (1994)                 66172
Shawshank Redemption, The (1994)    63366
Silence of the Lambs, The (1991)    63299
Jurassic Park (1993)                59715
Name: title, dtype: int64

In [21]:
df["title"].value_counts().tail() ## en az yorum alanlar

Copper Mountain (1983)                             1
Dr. Dolittle: Tail to the Chief (2008)             1
Cost of Oil: Voices from the Arctic, The (2009)    1
Blackbird (1959)                                   1
Always (1985)                                      1
Name: title, dtype: int64

In [22]:
## belli bir seviye üzeri (3000 ve üzeri) yorum yapılanları seçtim

In [23]:
choose_movies = pd.DataFrame(df["title"].value_counts())
great_movies = choose_movies[choose_movies["title"] >= 3000].index
great_movies = df[df["title"].isin(great_movies)]

In [24]:
great_movies.shape

(14970107, 6)

In [25]:
great_movies["title"].nunique()  ## kalan eşşiz film sayısı

1544

In [26]:
user_movie_df = great_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
## index e sürunları aldım, colonlarda isimler, values da rating olsun

In [27]:
user_movie_df.head(10)

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Things I Hate About You (1999),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),13 Going on 30 (2004),"13th Warrior, The (1999)",1984 (Nineteen Eighty-Four) (1984),2 Days in the Valley (1996),...,You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Zodiac (2007),Zombieland (2009),Zoolander (2001),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
movie_name = "Ocean's Twelve (2004)"   ##input("enter the Moive Name")
movie_name = user_movie_df[movie_name]
user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(5)

title
Ocean's Twelve (2004)                              1.000000
Ocean's Thirteen (2007)                            0.681654
Ocean's Eleven (2001)                              0.551280
Pirates of the Caribbean: At World's End (2007)    0.456309
Italian Job, The (2003)                            0.455345
dtype: float64

**Öneri Yapılacak Kullanıcının İzlediği Filmlerin Belirlenmesi**

In [29]:
random_user = int(pd.Series(user_movie_df.index).sample(1, random_state=45).values)

In [30]:
random_user_df = user_movie_df[user_movie_df.index == random_user]


In [31]:
random_user_df ## hedef kullanıcının değerlendirmeleri

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Things I Hate About You (1999),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),13 Going on 30 (2004),"13th Warrior, The (1999)",1984 (Nineteen Eighty-Four) (1984),2 Days in the Valley (1996),...,You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Zodiac (2007),Zombieland (2009),Zoolander (2001),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
46127.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist()

## izlenen filmler içinde NAN olmayanları seçtim


In [33]:
movies_watched

['Apocalypse Now (1979)',
 'Batman (1989)',
 'Braveheart (1995)',
 'Bridge on the River Kwai, The (1957)',
 'Butch Cassidy and the Sundance Kid (1969)',
 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)',
 'Fatal Attraction (1987)',
 'Gone with the Wind (1939)',
 "It's a Wonderful Life (1946)",
 'Lawrence of Arabia (1962)',
 'Liar Liar (1997)',
 'North by Northwest (1959)',
 'Patriot, The (2000)',
 'Philadelphia (1993)',
 'Pinocchio (1940)',
 'Shawshank Redemption, The (1994)',
 'Silence of the Lambs, The (1991)',
 'Snow White and the Seven Dwarfs (1937)',
 'Sound of Music, The (1965)',
 'Sting, The (1973)']

In [34]:
user_movie_df.loc[user_movie_df.index == random_user, user_movie_df.columns == "Schindler's List (1993)"]
len(movies_watched)  ## kaç tane film izlemiş 
## perc = 60/100*20, benzerlik oranı 12 olsun hedef kitle ile

20

**Aynı Filmleri İzleyen Diğer Kullanıcıların Verisine ve Id'lerine Erişmek**

In [35]:
movies_watched_df = user_movie_df[movies_watched]
user_movie_count = movies_watched_df.T.notnull().sum() ## izlenen filmlerin transposunu aldıktan sonra
# izleyip izlemediklerine göre toplam değeri bastırdım
user_movie_count = user_movie_count.reset_index()
## indexte yer alan user idleri değişkene çevirdim ve her bir ID nin yanında kaç tane film izlenmiş

In [36]:
movies_watched_df.head(5)
movies_watched_df.shape ## kaç tane kullanıcı ile ortak film izlenmiş

(138490, 20)

In [37]:
user_movie_count.columns = ["userId", "movie_count"]

In [38]:
perc = 60/100

In [39]:
user_movie_count[user_movie_count["movie_count"] > perc].sort_values("movie_count", ascending=False)
## 12 den fazla ortak film olan user'lar gelsin

,userId,movie_count
34853,34856.0,20
74051,74054.0,20
91864,91867.0,20
91190,91193.0,20
20130,20132.0,20
...,...,...
39047,39050.0,1
83549,83552.0,1
114654,114657.0,1
10166,10168.0,1


In [40]:
user_movie_count

,userId,movie_count
0,1.0,4
1,2.0,3
2,3.0,5
3,4.0,2
4,5.0,6
...,...,...
138485,138489.0,2
138486,138490.0,2
138487,138491.0,0
138488,138492.0,3


In [41]:
user_movie_count[user_movie_count["movie_count"] == 20].count() ## hedef userın izlediği total film sayısı 20, onunla birebir tüm filmleri izleyen kişi sayısı = 127

userId         127
movie_count    127
dtype: int64

In [42]:
users_same_movies = user_movie_count[user_movie_count["movie_count"] > perc]["userId"]


In [43]:
users_same_movies.head() ## %60dan fazla ortak olanlar gelsin 


0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
Name: userId, dtype: float64

In [44]:
users_same_movies.count()

113820

**Öneri Yapılacak Kullanıcı ile En Benzer Davranışlı Kullanıcıların Belirlenmesi**

In [45]:
final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies.index)],
                      random_user_df[movies_watched]])

## aynı filmi izleyen kullanıcıların indexlerini getirdim

In [46]:
final_df.head()

title,Apocalypse Now (1979),Batman (1989),Braveheart (1995),"Bridge on the River Kwai, The (1957)",Butch Cassidy and the Sundance Kid (1969),Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),Fatal Attraction (1987),Gone with the Wind (1939),It's a Wonderful Life (1946),Lawrence of Arabia (1962),Liar Liar (1997),North by Northwest (1959),"Patriot, The (2000)",Philadelphia (1993),Pinocchio (1940),"Shawshank Redemption, The (1994)","Silence of the Lambs, The (1991)",Snow White and the Seven Dwarfs (1937),"Sound of Music, The (1965)","Sting, The (1973)"
userId,,,,,,,,,,,,,,,,,,,,
1.0,3.5,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.5,NaN,NaN,NaN
2.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,5.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN
6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
final_df.shape

(113818, 20)

In [ ]:
final_df.T.corr()

In [ ]:
corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
corr_df = pd.DataFrame(corr_df, columns=["corr"])
corr_df.index.names = ['user_id_1', 'user_id_2']
corr_df = corr_df.reset_index()


# Hedef user ile yüzde 65 ve üzeri korelasyona sahip kullanıcılar:
top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] >= 0.65)][
    ["user_id_2", "corr"]].reset_index(drop=True)


top_users = top_users.sort_values(by='corr', ascending=False)
top_users.rename(columns={"user_id_2": "userId"}, inplace=True)

rating = pd.read_csv('datasets/movie_lens_dataset/rating.csv')
top_users_ratings = top_users.merge(rating[["userId", "movieId", "rating"]], how='inner')
